# 자료구조 3분 발표

### 1. 위도, 경도데이터와 최소힙을 사용한 우선순위 큐를 사용하여 근처 여행지 추리기

In [1]:
!pip install googlemaps

##### 구글 지도 api로 현재 위치의 위도와 경도 가져오기

In [152]:
import requests
import googlemaps
import json

GOOGLE_API_KEY = '키값 입력하는 곳(보안상 지움)'

url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={GOOGLE_API_KEY}'
data = {
    'considerIp': True, # 현 IP로 데이터 추출
}

result = json.loads(requests.post(url, data).text) # 해당 API에 요청을 보내며 데이터를 추출한다.

lat = result["location"]["lat"] # 현재 위치의 위도 추출
long = result["location"]["lng"] # 현재 위치의 경도 추출

present = (lat, long)

print(f'현재 위치의 위도와 경도: {present}')

현재 위치의 위도와 경도: (37.3793199, 127.1365699)


In [120]:
# 서울 가볼만한 곳 데이터 불러오기(https://github.com/gyunggyung/Sejong_ITIP-)
# 가져와서 전처리 조금 진행하였음
import pandas as pd

seoul_place_df = pd.read_csv("/Users/lwh/학교/3-1/자료구조/project/Seoul_Place.csv")

index = list(range(0,260))
seoul_place_df["index"] = index
seoul_place_df.head()

,Name,Function,Score,NumOfReviews,Longitude,Latitude,index
0,Gyeongbokgung Palace,역사적 명소,4.6,"12,779",126.974847,37.579617,0
1,Changdeokgung,역사적 명소,4.6,"3,657",126.988809,37.579291,1
2,Bukchon Hanok Village,역사적 명소,4.3,"4,848",126.981459,37.582408,2
3,Lotte World,놀이공원,4.2,"13,061",127.096922,37.511281,3
4,Namsan Park,공원,4.5,179,126.988705,37.550990,4


In [53]:
# 데이터에서 코스 종류 파악하기
seoul_place_df["Function"].value_counts()

카페        66
공원        43
역사적 명소    35
음식점       34
영화관       28
놀이공원      24
미술관       22
박물관        7
시장         4
Name: Function, dtype: int64

##### 위도와 경도로 거리 계산하기(haversine)
-   두 위경도(Latitude, Longitude)데이터의 거리를 구해야할 때 편리한 패키지

In [44]:
!pip install haversine

In [158]:
from haversine import haversine

def dist_cal(now, df):
    global dist_list
    dist_list = []
    # 위경도 입력
    for i in range(len(df)):
        compare = (df.iloc[i, 5], df.iloc[i, 4])

        # 거리 계산
        dist = haversine(now, compare, unit = 'km')
        dist_list.append(dist)

    df['dist'] =  dist_list

# 시작위치 지정
now = present
dist_cal(now, seoul_place_df)


##### 최소힙을 사용한 우선순위 큐
-   거리가 최소인 값만 찾으면 되므로 굳이 모든 것들을 완벽히 정렬할 필요가 없으므로 최소힙을 사용한 우선순위 큐를 사용함


-   파이썬의 heapq 라이브러리를 통해 손 쉽게 최소힙과 최대힙을 구현할 수 있다.
-   heapq는 기본적으로 최소힙으로 구현되어있다.
-   heapq의 heappush를 통해 값들을 삽입하면 해당 값들은 숫자가 가장 작은 순서대로 트리 구조로 값이 저장된다.
-   heapify 연산을 통해서 아무런 정렬이 되지 않은 리스트의 원소들을 힙으로 변환할 수 있다.

In [159]:
from heapq import heappush # 이 모듈은 우선순위 큐 알고리즘이라고도 하는 힙(heap) 큐 알고리즘의 구현을 제공합니다.
def sort_by_heap_q(df):
    global heap_q
    dist_list = df['dist']
    heap_q = []
    for idx, dist in enumerate(dist_list):
        heappush(heap_q, (dist, idx)) # push할때 튜플로 넣어주기, 튜플 첫번째 값(거리)으로 정렬해줌, 인덱스 살릴 수 있음
    

sort_by_heap_q(seoul_place_df)
print(f'전체 개수: {len(heap_q)}')
print(heap_q)

전체 개수: 260
[(9.884118282730954, 250), (12.799831169096967, 43), (10.43494591394306, 243), (17.1805197120786, 158), (13.647065904947441, 41), (13.14049931694818, 221), (11.688603553108361, 220), (19.573464228267408, 258), (17.677674789868984, 78), (15.085083424604965, 3), (14.841784678849024, 42), (15.592994200176843, 197), (13.155774561395067, 17), (15.732542230629617, 236), (12.123808798716224, 23), (19.86993980634939, 64), (19.86993980634939, 68), (19.56456895543939, 72), (18.497999128384116, 156), (15.59628498678245, 82), (15.793056600329288, 22), (16.695560088006022, 20), (19.505164686240946, 189), (19.688083227148166, 24), (16.509349158697592, 202), (16.695250277437385, 212), (13.165049227473878, 206), (19.86993980634939, 111), (17.74441280819947, 235), (19.86993980634939, 59), (14.702250100276487, 233), (19.86993980634939, 127), (19.86993980634939, 131), (19.86993980634939, 135), (19.86993980634939, 70), (19.86993980634939, 143), (19.86993980634939, 71), (19.18609382653012, 38), 

- 최소힙을 사용한 우선순위 큐를 사용한 정렬을 기준으로 기존 데이터프레임 다시 정렬

In [160]:
import pandas as pd
heap_q_dict = {'index': []}
for tup in heap_q:
    heap_q_dict['index'].append(tup[1]) # 인덱스 추가
    
heap_q_df = pd.DataFrame(heap_q_dict)

heap_sorted_df = pd.merge(seoul_place_df, heap_q_df, on='index', how='right')

print(f'전체 개수: {len(heap_sorted_df)}')
heap_sorted_df.head()

전체 개수: 260


,Name,Function,Score,NumOfReviews,Longitude,Latitude,index,dist
0,에버랜드,놀이공원,4.4,"16,677",127.167547,37.293910,250,9.884118
1,Garak Market,시장,4.0,"1,171",127.112674,37.492857,43,12.799831
2,남한산성유원지관리사무실,놀이공원,4.3,16,127.144275,37.472964,243,10.434946
3,CGV 강남,영화관,3.7,"1,640",127.017559,37.501565,158,17.180520
4,Yangjae Citizens' Forest,공원,4.4,486,127.033384,37.470686,41,13.647066


##### 최소값 출력
- heappop()
-  힙에서 원소를 삭제
- 원소를 삭제할 대상 리스트를 인자로 넘기면, 가장 작은 원소를 삭제 후에 그 값을 리턴


In [161]:
from heapq import heappop

min = heappop(heap_q)

print(f'min: {min}')

print(heap_q)

min: (9.884118282730954, 250)
[(10.43494591394306, 243), (12.799831169096967, 43), (11.688603553108361, 220), (17.1805197120786, 158), (13.647065904947441, 41), (13.14049931694818, 221), (12.123808798716224, 23), (19.573464228267408, 258), (17.677674789868984, 78), (15.085083424604965, 3), (14.841784678849024, 42), (15.592994200176843, 197), (13.155774561395067, 17), (15.732542230629617, 236), (14.702250100276487, 233), (19.86993980634939, 64), (19.86993980634939, 68), (19.56456895543939, 72), (18.497999128384116, 156), (15.59628498678245, 82), (15.793056600329288, 22), (16.695560088006022, 20), (19.505164686240946, 189), (19.688083227148166, 24), (16.509349158697592, 202), (16.695250277437385, 212), (13.165049227473878, 206), (19.86993980634939, 111), (17.74441280819947, 235), (19.86993980634939, 59), (16.024491349241526, 249), (19.86993980634939, 127), (19.86993980634939, 131), (19.86993980634939, 135), (19.86993980634939, 70), (19.86993980634939, 143), (19.86993980634939, 71), (19.1

In [162]:
# 다음 행성지 정보 출력
heap_sorted_df.iloc[0,0]

'에버랜드'

- 이 결과 같은 경우에는 지금 제가 살고 있는 분당 서현동 기준으로 되어있어서 조금 이상한 결과가 나왔습니다
- 아래에서는 시작점을 서울에 있는 호텔로 지정하여 실제 사용과 가깝게 진행해보겠습니다.

##### 1차 최종 구현
- 거리순으로 5개 여행지를 추리기

In [121]:
travel_place = []
def nearest_place(now, df, n):
    for i in range(n):
        # 위도와 경도로 거리 계산하기
        dist_cal(now, df)
        # 최소힙을 사용한 우선순위 큐
        sort_by_heap_q(df)
        # 최소힙을 사용한 우선순위 큐를 사용한 정렬을 기준으로 기존 데이터프레임 다시 정렬
        heap_q_dict = {'index': []}
        for tup in heap_q:
            heap_q_dict['index'].append(tup[1]) # 인덱스 추가 
        heap_q_df = pd.DataFrame(heap_q_dict) # 데이터 프레임으로 만들기
        heap_sorted_df = pd.merge(df, heap_q_df, on='index', how='right')
        # 다음 행성지 정보 출력 및 저장
        print(f'{i+1}번째 행선지: {heap_sorted_df.iloc[i,0]}')
        travel_place.append(heap_sorted_df.iloc[i,0])
        # 현재 위치 업데이트
        now = (heap_sorted_df.iloc[0, 5], heap_sorted_df.iloc[0, 4])
        # 최소값 pop
        min = heappop(heap_q)
        # 데이터프레임 업데이트(첫번째 행 제거, dist열 제거)
        heap_sorted_df = heap_sorted_df.drop(index=0, axis=0)
        heap_sorted_df.drop(columns=['dist'], axis=1)
        df = heap_sorted_df
    #결과출력
    print(f'추천된 여행지: {travel_place}')

In [122]:
# 데이터프레임 지정
df = seoul_place_df
# 여행지 개수 지정(반복횟수)
n = 5
# 시작위치 지정
now = (37.5558733, 127.0051289) # 서울 신라호텔

# 위도,경도 기준으로 거리순으로 여행지 추리기
nearest_place(now, df, 5)

1번째 행선지: 남산골한옥마을
2번째 행선지: 명동성당
3번째 행선지: 서울시립미술관 서소문본관
4번째 행선지: 반포한강공원
5번째 행선지: CGV 피카디리1958
추천된 여행지: ['남산골한옥마을', '명동성당', '서울시립미술관 서소문본관', '반포한강공원', 'CGV 피카디리1958']


# 개선방안

### 교수님 피드백

- 여행 경로를 빠르게 정리해주는 서비스에서 좌표를 기준으로 한 거리 순으로 정리하면 실제 이동 거리와 상당한 차이가 있을 텐데 이 부분이 고려가 된 건가요? 

- 그리고 이동 수단에 따른 경로도 다를 것 같습니다. 이 부분으로 추가적으로 생각해보길 바래요. 

### 피드백 반영 방향
- 대중교통 및 도보 이동까지 포함시키면 너무 복잡해지고 변수가 많아진다. 그래서 차로 가는 최소시간(최단거리)을 활용한 방문 순서를 결정해주는 프로그램을 만들기로 결정하였습니다.

In [125]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup as BS
from ordered_set import OrderedSet
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import random

In [126]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'

In [127]:
driver = Chrome('/Users/lwh/학교/3-1/비즈니스 프로그래밍/crawling/selenium/chromedriver.exe')

/var/folders/x8/jzrcwn_x1cz2qfy8mdnnc4sc0000gn/T/ipykernel_10591/1563853991.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('/Users/lwh/학교/3-1/비즈니스 프로그래밍/crawling/selenium/chromedriver.exe')


In [128]:
def get_time_dist(departure, arrival):
    global t, d

    # 카카오맵 접속
    driver.get("https://map.kakao.com/")
    time.sleep(random.randrange(1,6))
    # 길찾기 탭 클릭
    driver.find_element(By.XPATH, '//*[@id="search.tab2"]/a').click()
    time.sleep(random.randrange(1,6))
    # 출발지 입력후 엔터
    driver.find_element(By.XPATH, '//*[@id="info.route.waypointSuggest.input0"]').send_keys(departure+ Keys.ENTER)
    time.sleep(random.randrange(1,6))
    # 도착지 입력후 엔터
    driver.find_element(By.XPATH, '//*[@id="info.route.waypointSuggest.input1"]').send_keys(arrival+ Keys.ENTER)
    time.sleep(random.randrange(1,6))
    # 이동수단 '자동차'로 지정
    driver.find_element(By.XPATH, '//*[@id="cartab"]').click()
    time.sleep(random.randrange(1,6))

    # 이동 시간 및 거리 구해오기
    html = driver.page_source
    soup = BS(html, 'lxml')
    # 시간
    t = soup.select_one('span.time').text

    # 거리
    d = soup.select_one('span.distance').text

In [131]:
# 출발지 지정
departure = '아주대학교'
# 도착지 지정
arrival = '부산대학교'
get_time_dist(departure, arrival)

# 출력
print(f'시간: {t}')
print(f'거리: {d}')

시간: 4시간 15분
거리: 369.5km


In [132]:
def hour2min(travel_time):
    global travel_min
    travel_time = travel_time.split(' ')

    if len(travel_time) == 1:
        travel_min = travel_time[0][:-1]
    elif len(travel_time) == 2:
        travel_min = int(travel_time[0][:-2]) * 60 + int(travel_time[1][:-1])
    else:
        return "시간이 잘못된거 같은데...?"


In [193]:
#travel_place_list = ['Gyeongbokgung Palace', 'Lotte World', '최가커피', '와플그란데', '엉뚱식당 한우전문점']
#travel_place_list = travel_place
travel_place = ['남산골한옥마을', '명동성당', '서울시립미술관 서소문본관', '반포한강공원', 'CGV 피카디리1958']

In [142]:
travel_min_list = []
departure = '서울신라호텔'
for travel_place in travel_place_list:
    get_time_dist(departure, travel_place)
    hour2min(t)
    travel_min_list.append((int(travel_min), travel_place))

In [143]:
travel_min_list

[(7, '남산골한옥마을'),
 (10, '명동성당'),
 (20, '서울시립미술관 서소문본관'),
 (13, '반포한강공원'),
 (14, 'CGV 피카디리1958')]

##### 기수정렬
- 레코드를 비교하지 않고 분배하여 정렬을수행
- 시간복잡도가 O(dn)이므로 비교에 의한 정렬의 하한이 O(n logn)보다 좋을 수 있음
- 이동 소요 시간을 분으로 나타내면 정수이므로 이거로 비교하면 시간이 단축되고 연산량도 줄어들 것으로 예상되어 시도함

In [164]:
from collections import deque

def radix_sort(nums):
    buckets = [deque() for _ in range(10)]

    max_val = max(nums)
    Q = deque(nums)
    cur_ten = 1

    while max_val[0] >= cur_ten:
        while Q:
            num = Q.popleft()
            buckets[(num[0] // cur_ten) % 10].append(num)

        for bucket in buckets:
            while bucket:
                Q.append(bucket.popleft())

        cur_ten *= 10

    return list(Q)

In [163]:
print(radix_sort(travel_min_list))
print(f'다음 여행지는 {travel_min_list[0][1]}입니다.')

[(7, '남산골한옥마을'), (10, '명동성당'), (13, '반포한강공원'), (14, 'CGV 피카디리1958'), (20, '서울시립미술관 서소문본관')]
다음 여행지는 남산골한옥마을입니다.


In [180]:
def travel_order(travel_place_list, departure):
    for i in range(len(travel_place_list)):
        # 이동시간 분으로 구하기
        travel_min_list = []
        for travel_place in travel_place_list:
            get_time_dist(departure, travel_place)
            hour2min(t)
            travel_min_list.append((int(travel_min), travel_place))
        # 기수정렬하기
        radix_sort(travel_min_list)
        # 방문장소 출력하기
        print(f'{i+1}번째 여행지는 {travel_min_list[0][1]}입니다.')
        # 다음 방문할 장소 지우기
        del travel_place_list[0]


In [194]:
travel_place

['남산골한옥마을', '명동성당', '서울시립미술관 서소문본관', '반포한강공원', 'CGV 피카디리1958']

In [ ]:
# 여행지 리스트 지정
travel_place_list = travel_place
# 초기 출발 장소 지정
departure = '서울신라호텔'
# 최종 결과물!!
travel_order(travel_place_list, departure)